In [ ]:
import os, csv, multiprocessing, subprocess, re, shutil
from pathlib import Path
from google.colab import files

# ---------- 1) Setup & Tools ----------
print("📦 Installing Tools (Vina + Open Babel)...")
os.system("apt-get -qq update && apt-get -qq install -y autodock-vina openbabel > /dev/null")

# ---------- 2) Helpers & Auto-Box Logic ----------
def ask_float(prompt: str, default: float) -> float:
    try:
        s = input(f"{prompt} [{default}]: ").strip()
        return float(s) if s else float(default)
    except: return float(default)

def ask_int(prompt: str, default: int) -> int:
    try:
        s = input(f"{prompt} [{default}]: ").strip()
        return int(s) if s else int(default)
    except: return int(default)

def detect_ligand_bbox(pdb_path: str):
    ALLOW_WAT = {"HOH","WAT","OH2","H2O","DOD"}
    ALLOW_IONS = {"ZN","MG","MN","FE","CA","NA","K","CL","CO","CU","NI","CD"}
    groups = {}
    with open(pdb_path, "r") as f:
        for raw in f:
            if not raw.startswith("HETATM"): continue
            resn = raw[17:20].strip()
            if resn in ALLOW_WAT or resn in ALLOW_IONS: continue
            chain, resid = raw[21:22].strip() or "-", raw[22:26].strip()
            try:
                coords = (float(raw[30:38]), float(raw[38:46]), float(raw[46:54]))
                groups.setdefault((resn, chain, resid), []).append(coords)
            except: continue
    if not groups: return None
    key = max(groups.keys(), key=lambda k: len(groups[k]))
    pts = groups[key]
    xs, ys, zs = zip(*pts)
    return (min(xs), min(ys), min(zs)), (max(xs), max(ys), max(zs)), key

# ---------- 3) Inputs & File Uploads ----------
print("\n📤 Upload Receptor PDB (e.g., protein.pdb):")
uploaded_rec = files.upload()
rec_pdb = list(uploaded_rec.keys())[0]

print("\n📤 Upload SMILES Library (.smi or .txt):")
uploaded_smi = files.upload()
smi_file = list(uploaded_smi.keys())[0]

# ---------- 4) Parameter Configuration ----------
auto = detect_ligand_bbox(rec_pdb)
if auto:
    (mnx,mny,mnz), (mxx,mxy,mxz), meta = auto
    cx, cy, cz = (mnx+mxx)/2, (mny+mxy)/2, (mnz+mxz)/2
    sx, sy, sz = max(20.0, (mxx-mnx)+8), max(20.0, (mxy-mny)+8), max(20.0, (mxz-mnz)+8)
    print(f"\n📍 Auto-detected Co-crystal Ligand: {meta}")
else:
    cx = cy = cz = 0.0; sx = sy = sz = 22.0

print("\n🛠️ SET DOCKING PARAMETERS:")
cx, cy, cz = ask_float("  Center X", cx), ask_float("  Center Y", cy), ask_float("  Center Z", cz)
sx, sy, sz = ask_float("  Size X", sx), ask_float("  Size Y", sy), ask_float("  Size Z", sz)
EXHAUST = ask_int("  Exhaustiveness (Search intensity; 8 is standard)", 8)
pH = ask_float("  Protonation pH", 7.4)

# ---------- 5) Parallel Preparation ----------
def prep_ligand(args):
    idx, smi = args
    name = f"lig_{idx:04d}"
    out_pdbqt = f"ligands/{name}.pdbqt"
    temp_smi = f"temp_{idx}.smi"
    with open(temp_smi, "w") as f: f.write(smi)
    # Generate 3D and protonate
    subprocess.run(f'obabel -ismi {temp_smi} -opdbqt --gen3d -p {pH} -O {out_pdbqt} > /dev/null 2>&1', shell=True)
    if os.path.exists(temp_smi): os.remove(temp_smi)
    return out_pdbqt

print(f"\n🧪 Generating 3D PDBQTs for the library...")
Path("ligands").mkdir(exist_ok=True)
with open(smi_file, "r") as f:
    smiles_list = [(i, line.strip()) for i, line in enumerate(f) if line.strip()]

with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
    pool.map(prep_ligand, smiles_list)

# ---------- 6) Receptor Preparation & Execution ----------
print("🧬 Preparing Receptor...")
os.system(f'obabel -ipdb "{rec_pdb}" -opdbqt -xr -O protein.pdbqt > /dev/null 2>&1')

print(f"🚀 Running Vina Batch (Library size: {len(smiles_list)})...")
Path("results").mkdir(exist_ok=True)
vina_cmd = (
    f'vina --receptor protein.pdbqt --batch ligands/*.pdbqt '
    f'--center_x {cx:.3f} --center_y {cy:.3f} --center_z {cz:.3f} '
    f'--size_x {sx:.3f} --size_y {sy:.3f} --size_z {sz:.3f} '
    f'--exhaustiveness {EXHAUST} --dir results'
)
os.system(vina_cmd)

# ---------- 7) Data Consolidation & Zip ----------
print("\n📊 Aggregating and Compressing Results...")
summary = []
for out_file in Path("results").glob("*.pdbqt"):
    with open(out_file, "r") as f:
        for line in f:
            if "REMARK VINA RESULT:" in line:
                summary.append([out_file.stem, float(line.split()[3])])
                break

summary.sort(key=lambda x: x[1]) # Sort by best energy

# Save CSV
with open("screening_results.csv", "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["Ligand_ID", "Binding_Affinity_kcal_mol"])
    w.writerows(summary)

# Zip the results folder
shutil.make_archive("all_docking_results", 'zip', "results")

print(f"\n✅ Finished!")
print(f"🥇 Best hit: {summary[0][0]} ({summary[0][1]} kcal/mol)")
files.download("screening_results.csv")
files.download("all_docking_results.zip")

📦 Installing Tools (Vina + Open Babel)...

📤 Upload Receptor PDB (e.g., protein.pdb):


Saving Abl1.pdb to Abl1.pdb

📤 Upload SMILES Library (.smi or .txt):


Saving test_library.txt to test_library.txt

📍 Auto-detected Co-crystal Ligand: ('9DP', 'A', '1')

🛠️ SET DOCKING PARAMETERS:
  Center X [19.703500000000002]: 19.793
  Center Y [22.3555]: 22.314
  Center Z [10.6975]: 10.792
  Size X [20.0]: 20
  Size Y [20.0]: 20
  Size Z [20.0]: 20
  Exhaustiveness (Search intensity; 8 is standard) [8]: 2
  Protonation pH [7.4]: 7.4

🧪 Generating 3D PDBQTs for the library...
🧬 Preparing Receptor...
🚀 Running Vina Batch (Library size: 13)...
